In [67]:
# !pip install ccxt
# !pip install slacker
import ccxt
import pprint
from datetime import datetime
from slacker import Slacker

In [71]:
# 파일로부터 apiKey, Secret 읽기 
with open("apiKey_ham.txt") as f:
    lines = f.readlines()
    apiKey = lines[0].strip() 
    secret = lines[1].strip() 
    slackToken = lines[2].strip()

In [74]:
slack = Slacker(slackToken)
binance = ccxt.binance()

In [ ]:
# 슬랙 메세지 보내기
# slack.chat.post_message('#autostock','hi bum!')

In [ ]:
# 티커들 조회
markets = binance.fetch_tickers()
print(markets.keys())

In [84]:
# 현재가 조회
ticker = binance.fetch_ticker('ETH/USDT')
print(ticker['open'], ticker['high'], ticker['low'], ticker['close'])

3854.69 3925.95 3752.0 3796.35


In [20]:
# 과거 데이터 조회
ohlcvs = binance.fetch_ohlcv('ETH/USDT')
# for ohlc in ohlcvs:
#     print(datetime.fromtimestamp(ohlc[0]/1000).strftime('%Y-%m-%d %H:%M:%S'))

---

In [75]:
# 선물 잔고 조회
# free : 보유중, used : 거래진행중, 
binance = ccxt.binance({
    'apiKey': apiKey,
    'secret': secret,
    'enableRateLimit': True,
    'options': {
        'defaultType': 'future'
    }
})
balance = binance.fetch_balance(params={"type": "future"})
print(balance['USDT'])

{'free': 80.2898479, 'used': 0.76103315, 'total': 80.967141}


---

In [19]:
def showDateTime(dateTime):
    return datetime.fromtimestamp(dateTime/1000).strftime('%Y-%m-%d %H:%M:%S')

In [85]:
# ohlcv
eth5m = binance.fetch_ohlcv(
    symbol="ETH/USDT", 
    timeframe='5m', 
    since=None, 
    limit=10)
for e in eth5m:
    print(showDateTime(e[0]), e[1:])

2021-10-18 01:35:00 [3815.74, 3816.36, 3807.79, 3808.0, 3669.59]
2021-10-18 01:40:00 [3808.01, 3810.68, 3804.0, 3807.93, 3828.246]
2021-10-18 01:45:00 [3807.93, 3811.69, 3801.18, 3802.53, 2586.143]
2021-10-18 01:50:00 [3802.53, 3811.63, 3802.09, 3810.37, 2555.941]
2021-10-18 01:55:00 [3810.35, 3815.27, 3808.21, 3813.88, 3432.268]
2021-10-18 02:00:00 [3813.81, 3813.81, 3806.1, 3808.99, 2615.307]
2021-10-18 02:05:00 [3808.99, 3809.39, 3802.0, 3802.98, 2110.707]
2021-10-18 02:10:00 [3802.99, 3803.1, 3798.67, 3802.65, 2398.536]
2021-10-18 02:15:00 [3802.65, 3802.83, 3791.83, 3797.63, 3915.485]
2021-10-18 02:20:00 [3797.63, 3798.78, 3793.23, 3796.01, 1821.419]


---

In [33]:
# 매수 매도
order = binance.create_market_buy_order(
    symbol="ETH/USDT",
    amount=0.002, 
)
pprint.pprint(order)
 
# order = binance.create_market_sell_order(
#     symbol="ETH/USDT",
#     amount=0.002, 
# )
# pprint.pprint(order)


{'amount': 0.002,
 'average': 3849.86,
 'clientOrderId': 'x-xcKtGhcu27a24d72355b4a0c810ef1',
 'cost': 7.69972,
 'datetime': None,
 'fee': None,
 'fees': [],
 'filled': 0.002,
 'id': '8389765509100861083',
 'info': {'avgPrice': '3849.86000',
          'clientOrderId': 'x-xcKtGhcu27a24d72355b4a0c810ef1',
          'closePosition': False,
          'cumQty': '0.002',
          'cumQuote': '7.69972',
          'executedQty': '0.002',
          'orderId': '8389765509100861083',
          'origQty': '0.002',
          'origType': 'MARKET',
          'positionSide': 'BOTH',
          'price': '0',
          'priceProtect': False,
          'reduceOnly': False,
          'side': 'BUY',
          'status': 'FILLED',
          'stopPrice': '0',
          'symbol': 'ETHUSDT',
          'timeInForce': 'GTC',
          'type': 'MARKET',
          'updateTime': '1634482056016',
          'workingType': 'CONTRACT_PRICE'},
 'lastTradeTimestamp': None,
 'postOnly': False,
 'price': 3849.86,
 'remaining

In [78]:
# 현재 포지션 확인
balance = binance.fetch_balance()
position = [x for x in balance['info']['positions'] if x['symbol'] == "ETHUSDT"][0]
print(position)

{'symbol': 'ETHUSDT', 'initialMargin': '0.76107348', 'maintMargin': '0.03805367', 'unrealizedProfit': '-0.08898516', 'positionInitialMargin': '0.76107348', 'openOrderInitialMargin': '0', 'leverage': '10', 'isolated': True, 'entryPrice': '3849.86', 'maxNotional': '4000000', 'positionSide': 'BOTH', 'positionAmt': '0.002', 'notional': '7.61073483', 'isolatedWallet': '0.76668155', 'updateTime': '1634486400385'}


In [83]:
binance.trades

{}

In [60]:
# 현재 포지션 2퍼 위에 매도 예약
# targetPrice = round(float(position["entryPrice"]) * 1.0022, 2)
# print(targetPrice)
# binance.create_limit_sell_order(
#     symbol = 'ETH/USDT',
#     amount = 0.002,
#     price = targetPrice
# )

3858.33


{'info': {'orderId': '8389765509101346200',
  'symbol': 'ETHUSDT',
  'status': 'NEW',
  'clientOrderId': 'x-xcKtGhcu41680a6da6274810a76ddb',
  'price': '3858.33',
  'avgPrice': '0.00000',
  'origQty': '0.002',
  'executedQty': '0',
  'cumQty': '0',
  'cumQuote': '0',
  'timeInForce': 'GTC',
  'type': 'LIMIT',
  'reduceOnly': False,
  'closePosition': False,
  'side': 'SELL',
  'positionSide': 'BOTH',
  'stopPrice': '0',
  'workingType': 'CONTRACT_PRICE',
  'priceProtect': False,
  'origType': 'LIMIT',
  'updateTime': '1634483299501'},
 'id': '8389765509101346200',
 'clientOrderId': 'x-xcKtGhcu41680a6da6274810a76ddb',
 'timestamp': 1634483299501,
 'datetime': '2021-10-17T15:08:19.501Z',
 'lastTradeTimestamp': None,
 'symbol': 'ETH/USDT',
 'type': 'limit',
 'timeInForce': 'GTC',
 'postOnly': False,
 'side': 'sell',
 'price': 3858.33,
 'stopPrice': None,
 'amount': 0.002,
 'cost': 0.0,
 'average': None,
 'filled': 0.0,
 'remaining': 0.002,
 'status': 'open',
 'fee': None,
 'trades': [],
 

In [58]:
# 대기 주문 조회
open_orders = binance.fetch_open_orders(
    symbol="ETH/USDT"
)
pprint.pprint(open_orders)

[{'amount': 0.002,
  'average': None,
  'clientOrderId': 'x-xcKtGhcuc3764ad988599b9c5d64a3',
  'cost': 0.0,
  'datetime': '2021-10-17T15:07:53.478Z',
  'fee': None,
  'fees': [],
  'filled': 0.0,
  'id': '8389765509101328129',
  'info': {'avgPrice': '0',
           'clientOrderId': 'x-xcKtGhcuc3764ad988599b9c5d64a3',
           'closePosition': False,
           'cumQuote': '0',
           'executedQty': '0',
           'orderId': '8389765509101328129',
           'origQty': '0.002',
           'origType': 'LIMIT',
           'positionSide': 'BOTH',
           'price': '3858.33',
           'priceProtect': False,
           'reduceOnly': False,
           'side': 'SELL',
           'status': 'NEW',
           'stopPrice': '0',
           'symbol': 'ETHUSDT',
           'time': '1634483273478',
           'timeInForce': 'GTC',
           'type': 'LIMIT',
           'updateTime': '1634483273478',
           'workingType': 'CONTRACT_PRICE'},
  'lastTradeTimestamp': None,
  'postOnly': Fal

In [59]:
# 모든 대기 주문 취소
binance.cancel_all_orders(symbol = "ETH/USDT")

{'code': '200', 'msg': 'The operation of cancel all open order is done.'}